I am using the late submission budget

# ASSIGNMENT 1                                                McGill:COMP767 - FALL 2019 
Student name (ID) 260571481

In [7]:
import numpy as np
import networkx as nx
from collections import OrderedDict
from utils import tensor_mult
from numpy import ndenumerate

### Problem 1. (5 points)

- __a__: 2 points) Derive the __Markov__ inequality below for a positive discrete random variable
(_Hint:_ rearrange to prove $a P(x \geq a) \leq \mathbb{E}[X]$ and substitute the definition of expectation.)
$$P(x > a) \leq \frac{\mathbb{E}[X]}{a}$$
- __b__: 3 points) Using this inequality prove the following, known as __Chebyshev__ inequality:
$$P(|X - \mathbb{E}[X]| > a) < \frac{Var(X)}{a^2}$$

### Solution
__a__) 

From definition of expected value we have:

$$\mathbb{E}(X) = \int_{-\infty}^\infty x f(x) dx$$
and if $X$ is non-negative random variable, we have:

$$\int_{-\infty}^\infty x f(x) dx = \int_0^\infty xf(x)dx$$

Hence from the derivation we have

$$\mathbb{E}(X) = \int_{0}^a x f(x) dx + \int_a^\infty xf(x)dx \geq \int_a^\infty xf(x)dx \geq a\int_a^\infty f(x)dx = a \mathbb{P}(X \geq a)$$

and given this we have

$$\mathbb{P}(x > a) \leq \frac{\mathbb{E}[X]}{a}$$


__b__) 

By the definition of variance we have :
$$\text{Var}(X) = \mathbb{E}[(X - \mathbb{E}(X))^2]$$

Hence we can write the given equation as following as the expected value in the numerator represents the expected value

$$\mathbb{P}((X - \mathbb{E}[X])^2 > a^2) < \frac{Var(X)}{a^2}$$


using Markov's inequality this is as now the denominator $a^2$ is now constant

### Problem 2. (5 points)

In showing that factorization in a Markov Network leads to local CIs we used the
following fact. Prove it using the definition of conditional independence:

$$
P \models (X \perp Y \mid Z) \quad \Leftrightarrow \quad P(X=x, Y=y, Z=z) = f(x, z)g(y,z) 
$$


### Solution

- $\Rightarrow$) 2 points 
The left side of this equivalence can be represented by following
$$\frac{\phi_1(X,Z) \phi_2(Y,Z)}{C}$$

where $C$ is normalizaing constant : $C := \sum_{x,y,z} \phi_1(x,z)\phi_2(y,z)$

Let 

1. $f(x,z) = \frac{\phi_1(x,z)}{\sum_{x,y,z} \phi_1(x,z)\phi_2(y,z)} = \frac{\phi_1(x,z)}{C}$
2. $g(y,z) = \phi_2(y,z)$


- $\Leftarrow$) 3 points

We have due to conditional probability:

$$\mathbb{P}(X = x, Y = y, Z= z) = \mathbb{P}(X = x , Y = y | Z = z)\mathbb{P}(Z = z)$$

hence given the right side of equation we have the following

$$\mathbb{P}(X = x, Y = y, Z= z) =\mathbb{P}(X = x , Y = y | Z = z)\mathbb{P}(Z = z) = f(x, z)g(y,z) $$

and 

$$\mathbb{P}(X = x , Y = y | Z = z) = \frac{f(x, z)g(y,z)}{\mathbb{P}(Z = z)} $$

If we let $\phi_1(X=x,Z=z) = \frac{f(x,z)}{\mathbb{P}(Z=z)}$ , and $\phi_2(Y=y, Z=z) = g(y,z)$, then we have

$$\mathbb{P}(X = x , Y = y | Z = z) = \frac{\phi_1(X=x,Z=z)\phi_2(Y=y, Z=z)}{C} = \frac{\phi_1(X,Z) \phi_2(Y,Z)}{C}$$

where in our case we can have $C = 1$ which represents our factorization of Markov Network $P \models (X \perp Y \mid Z)$.


### Problem 3. (10 points)

Here, we want to represent the joint probability $P(D,I,G,S,L)$ and answer arbitrary queries such as $P(D,I \mid G=0, L=1)$ for the running example below that we used extensively in the class.
<img src="./files/3_4.png" width="400">
For this, we need to define CPTs. We use the ```networkx``` package to represent a DAG and add CPTs as node attributes. the CPT for a node with $K$ parents is a $K+1$ dimensional array, where the first dimension is for the child node and the order of parents follows their order when the method ```DAG.predecessors(node)``` is called. This is the order in which the corresponding edges are added. 

Your task is to write the body of the function ```Pr()``` that calculates the array of the posterior marginal, given a DAG -- e.g., $P(D, L \mid G= 2, I = 1)$. 
For your implementation you can use the ```tensor_mult``` helper function provided in ```utility.py```. 

You can try implementing this function in three steps:

- calculate the joint PMF
- condition on the evidence (e.g., by multiplying the joint array with appropriate tensor of 0s and 1s.
- marginalize and normalize the final results (normalization might be necessary depending on your implementation of conditioning on the evidence)

There are more efficient ways of calculating the posterior marginals that we study in the __inference__ lectures.

### Solution

In [8]:
def sum_over_index(n, P_matrix):
    
    out = np.empty(P_matrix.shape[:n] + P_matrix.shape[n+1:])
    for keys,values in np.ndenumerate(out):
        p = 0
        for ii in range(0, P_matrix.shape[n]):
            p += P_matrix[keys[:n] + (ii,) + keys[n:]]
        out[keys] = p
    
    return out
        

In [61]:

# creating the BayesNet
BN = nx.DiGraph()
BN.add_node('D', cpt=np.array([.6,.4]))
BN.add_node('I', cpt=np.array([.7,.3]))

#a 3-dimensional array of shape 3x2x2 representing P(G|I,D).  
#note that the order of parents matters, here we have made sure the order is the same as
#the order returned by BN.predecessors('G')
BN.add_node('G', cpt=np.array([[[.3,.05],[.9,.5]],[[.4,.25],[.08,.3]],[[.3,.7],[.02,.2]]]))
BN.add_node('L', cpt=np.array([[.1,.4,.99],[.9,.6,.01]]))
BN.add_node('S', cpt=np.array([[.95,.2],[.05,.8]]))

# adding edges (note that the order of I,D -> G is important)
BN.add_edge('I','G')
BN.add_edge('D','G')
BN.add_edge('G', 'L')
BN.add_edge('I', 'S')

print("Is this a DAG? {}".format(nx.is_directed_acyclic_graph(BN)))
# we can use topological sort to get a topological ordering of nodes. What is the complexity of this sorting op.?
list(nx.topological_sort(BN))


def Pr(target, 
       evidence, # a dictionary of observations to conditin on -- eg, {'L':0, 'I':1}
       DAG #DAG containing the CPTs (BN above)
      ):

    # >>>> YOUR CODE HERE >>>>>>>
        #1. calculate joint pmf
    ## gives |G| = 3, |I| = 2, |D| = 2 in this order
    P_G_I_D = tensor_mult(tensor_mult(BN.node["G"]["cpt"], BN.node["I"]["cpt"], [1], [0]), 
                          BN.node["D"]["cpt"], [2],[0])
    P_G_I_D_S = tensor_mult(P_G_I_D, BN.node["S"]["cpt"], [1],[1])
    joint= tensor_mult(P_G_I_D_S, BN.node["L"]["cpt"], [0],[1])    
    
    #final joint order (|G|, |I|,|D|, |S|, |L| ) 
    
    # calculate denominator
    letter_index = {"G" : 0 , "I" : 1, "D" : 2, "S" : 3, "L" : 4}
    
    evidence_list = ["G", "I", "D", "S", "L"]
    Q = joint
    for i in BN.nodes():
            if i not in evidence.keys():
                Q = sum_over_index(evidence_list.index(i), Q)
                evidence_list.remove(i)

    denominator_list = []
    
    for i in evidence_list:
        denominator_list.append((i, letter_index[i], evidence[i]))
    
    denominator_list.sort(key=lambda tup: tup[1])  
    
    denominator_key = tuple([i[2] for i in denominator_list])
    
    denominator = Q[denominator_key]
    

    numerator_index = BN.nodes() - evidence.keys() - set(target)
    marginal_list = ["G", "I", "D", "S", "L"]
    for i in numerator_index:
        joint = sum_over_index(marginal_list.index(i), joint)
        marginal_list.remove(i)
    
    #calculate numerator
    numerator_shape = []    
    
    for i in target:
        if i == "G":
            numerator_shape.append(3)
        else:
            numerator_shape.append(2)
            
    marginal = np.zeros(tuple(numerator_shape))

    marginal_index = []
    for i in target:
        marginal_index.append((0, letter_index[i], i))
    for i in evidence:
        marginal_index.append((evidence[i], letter_index[i], i))
        

    marginal_index.sort(key = lambda tup : tup[1])
    find_non_target_index = [marginal_index.index(i) for i in marginal_index if i[2] in target]
    for keys,values in ndenumerate(marginal):
        list_keys = list(keys)
        for i,j in zip(list_keys, find_non_target_index):

            temp = list(marginal_index[j])
            temp[0] = i

            marginal_index[j] = temp

        A = tuple([i[0] for i in marginal_index])
        
        print(A, "A")
        print(joint.shape, "joint shape")
        marginal[keys] = joint[A]

    # >>>> YOUR CODE HERE >>>>>>>
    
    marginal /= marginal.sum()
    
    return marginal


Is this a DAG? True


### Problem 4. (10 points)

Your task in this assignment is to implement __D-separation__ algorithm for a DAG representation, similar to what we used in the previous problem. Note that (assuming non-deterministic factors) D-separation does not need access to the CPTs. The following function returns ```True``` if the given CI holds and ```False``` otherwise.

In [62]:
def is_cond_independent(X, #non-empty list of nodes -- e.g., ['I', 'D'] 
                        Y, #non-empty list of nodes. It has no intersection with X -- e.g., ['S']
                        Z, #list of nodes -- e.g., []
                        DAG #networkx DAG -- e.g., BN defined above
                       ):
    
    # >>>> YOUR CODE HERE >>>>>>>
        #phase one
        L = set(Z)
        A = []
        while len(L) != 0:
            temp = list(L)[0]
            L = set(list(L)[1:])
            if temp not in A:
                L = set(list(L) + list(DAG.predecessors(temp)))
            A = set(list(A) + [temp])
        #phase two

        L = [(X[0], "up")]
        V = []
        R = []
        while len(L) !=0:
            temp = list(set(L))[0]
            L = set(list(L)[1:])
            if temp not in V:
                if temp[0] not in set(Z):
                    R = set(list(R) + [temp[0]])
                V = set(list(V) + [temp])
                if temp[1] == "up" and temp[0] not in set(Z):
                    for z in DAG.predecessors(temp[0]):
                        L = set(list(L) + [(z[0], "up")])
                    for z in DAG.successors(temp[0]):
                        L = set(list(L) + [(z[0], "down")])
                elif temp[1] == "down":
                    if temp[0] not in list(Z):
                        for z in DAG.successors(temp[0]):
                            L = set(list(L) + [(z[0], "down")])
                    if temp[0] in A:
                        for z in DAG.predecessors(temp[0]):
                            L = set(list(L) + [(z[0], "up")])

        if Y[0] in R:
            return False
        else:
            return True
    # >>>> YOUR CODE HERE >>>>>>>

Let us verify that the CIs that we get from D-separation, match the definition of conditional independence using the conditional probabilities that we can numerically calculate using your solution to the problem 3. 
In the following we look at all queries in the form of $P \overset{?}{\models} {D} \perp {S} \mid \mathbf{Z} = \mathbf{0}$, where $\mathbf{Z} \subseteq \{I,G,L\}$ may contain any of the remaining variables. 

In [63]:
domains = {'I':[0,1], 'L':[0,1], 'G':[0,1,2]}
for Z in [{},{'I':0}, {'G':0}, {'L':0}, {'I':0,'G':0}, {'I':0,'L':0}, {'L':0,'G':0}, {'I':0,'G':0,'L':0}]:
    #conditional independence from D-separation
    CI_alg = is_cond_independent(['D'], ['S'], Z.keys(), BN)
    #conditional independence according to conditional probabilities
    CI_num = np.max(np.abs(Pr(['D','S'], Z, BN) - np.outer(Pr(['D'], Z, BN),Pr(['S'], Z, BN)))) < 1e-10
    #they should match
    assert(CI_num == CI_alg)
print("passed the minimal test")

[0, 1]
[(0, 2, 'D'), (0, 3, 'S')]
[0, 0] list_keys
[0, 1] find_non_target_index
[0, 2, 'D'] temp
[0, 3, 'S'] temp
(0, 0) A
(2, 2) joint shape
[0, 1] list_keys
[0, 1] find_non_target_index
[0, 2, 'D'] temp
[1, 3, 'S'] temp
(0, 1) A
(2, 2) joint shape
[1, 0] list_keys
[0, 1] find_non_target_index
[1, 2, 'D'] temp
[0, 3, 'S'] temp
(1, 0) A
(2, 2) joint shape
[1, 1] list_keys
[0, 1] find_non_target_index
[1, 2, 'D'] temp
[1, 3, 'S'] temp
(1, 1) A
(2, 2) joint shape
[0]
[(0, 2, 'D')]
[0] list_keys
[0] find_non_target_index
[0, 2, 'D'] temp
(0,) A
(2,) joint shape
[1] list_keys
[0] find_non_target_index
[1, 2, 'D'] temp
(1,) A
(2,) joint shape
[0]
[(0, 3, 'S')]
[0] list_keys
[0] find_non_target_index
[0, 3, 'S'] temp
(0,) A
(2,) joint shape
[1] list_keys
[0] find_non_target_index
[1, 3, 'S'] temp
(1,) A
(2,) joint shape
[1, 2]
[(0, 1, 'I'), (0, 2, 'D'), (0, 3, 'S')]
[0, 0] list_keys
[1, 2] find_non_target_index
[0, 2, 'D'] temp
[0, 3, 'S'] temp
(0, 0, 0) A
(2, 2, 2) joint shape
[0, 1] list_k